In [1]:
import os
import sys
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'
sys.path.insert(0, '../scripts')

In [2]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Activation, Dropout
from mfcc_feature_extraction import MFCCFeatureExtraction
from load_file import LoadFile

In [3]:
tracks, labels = LoadFile('../assets/csv/test.csv').__getitem__()
test_features, test_labels = MFCCFeatureExtraction(tracks, labels).featuredata()
tracks, labels = LoadFile('../assets/csv/dev.csv').__getitem__()
dev_features, dev_labels = MFCCFeatureExtraction(tracks, labels).featuredata()
tracks, labels = LoadFile('../assets/csv/train.csv').__getitem__()
train_features, train_labels = MFCCFeatureExtraction(tracks, labels).featuredata()

In [4]:
dense_layer =  tf.keras.Sequential([
                                      Dense(100, input_shape =(20,) ),
                                      Activation('relu'),
                                      Dropout(0.5),

                                      Dense(200),
                                      Activation('relu'),
                                      Dropout(0.5),
                                    
                                      Dense(100),
                                      Activation('relu'),
                                      Dropout(0.5),
                                    
                                      Dense(5),
                                      Activation('softmax')
])

dense_layer.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 100)               2100      
_________________________________________________________________
activation (Activation)      (None, 100)               0         
_________________________________________________________________
dropout (Dropout)            (None, 100)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 200)               20200     
_________________________________________________________________
activation_1 (Activation)    (None, 200)               0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 200)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 100)               2

In [5]:
from datetime import datetime
from tensorflow.keras.callbacks import ModelCheckpoint

dense_layer.compile(loss = 'categorical_crossentropy', optimizer='adam', metrics = ['accuracy'])
num_epochs = 90
num_batch_size = 32

checkpointer = ModelCheckpoint(filepath = '../Checkpoints/audio_dense.hdf5', verbose = 1, save_best_only = True)

start = datetime.now()
dense_layer.fit(train_features, train_labels,batch_size = num_batch_size, epochs = num_epochs, validation_data = (dev_features, dev_labels), callbacks = [checkpointer])
end = datetime.now()

print("Training completed in time:", end - start)

Epoch 1/90
134/141 [===========================>..] - ETA: 0s - loss: 11.3207 - accuracy: 0.2673
Epoch 00001: val_loss improved from inf to 1.29378, saving model to ../Checkpoints/audio_dense.hdf5
141/141 [==============================] - 0s 3ms/step - loss: 10.9599 - accuracy: 0.2651 - val_loss: 1.2938 - val_accuracy: 0.4160
Epoch 2/90
124/141 [=========================>....] - ETA: 0s - loss: 2.1958 - accuracy: 0.3478
Epoch 00002: val_loss improved from 1.29378 to 1.11745, saving model to ../Checkpoints/audio_dense.hdf5
141/141 [==============================] - 0s 2ms/step - loss: 2.1290 - accuracy: 0.3456 - val_loss: 1.1174 - val_accuracy: 0.5800
Epoch 3/90
138/141 [============================>.] - ETA: 0s - loss: 1.3154 - accuracy: 0.4284
Epoch 00003: val_loss improved from 1.11745 to 0.90027, saving model to ../Checkpoints/audio_dense.hdf5
141/141 [==============================] - 0s 2ms/step - loss: 1.3118 - accuracy: 0.4282 - val_loss: 0.9003 - val_accuracy: 0.6120
Epoch 4/9